# Reporting

In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely import wkb
from dotenv import load_dotenv
import awswrangler as wr
import os
import yaml
import boto3
from IPython import display as ICD

In [ ]:
# Configs
# load environment variables with
load_dotenv()

# import config
with open("../config.yaml") as f:
    config = yaml.safe_load(f)

# connect to AWS with credentials
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("REGION")
BUCKET = os.getenv("BUCKET")
FOLDER = config["aws"]["folder"]

# connect to AWS
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)
s3 = session.client("s3")

In [ ]:
# S1: Get User Data
# get users data from S3
gdf_users = (
    wr.s3.read_parquet(
        f"s3://{BUCKET}/{FOLDER}/{config['aws']['users-file']}"
    )
    .rename(columns={"client_latitude": "latitude", "client_longitude": "longitude"})
    # convert to GeoDataFrame
    .pipe(gpd.GeoDataFrame)
    .assign(
        geometry=lambda x: gpd.points_from_xy(x['longitude'], x['latitude'])
    )
    .set_crs(epsg=4326)
)

# S2: Wrangle Data
# eliminate invalid values of raw_sim_operator_name
non_valid_operators = config['report']['non-valid-operators']
gdf_users = gdf_users[~gdf_users["raw_sim_operator_name"].isin(non_valid_operators)]

# standarize the name of Megacable
gdf_users.loc[
    gdf_users["raw_sim_operator_name"].isin(["Mega 4.5G", "Mega4.5G"]), "raw_sim_operator_name"
    ] = "Megacable"

In [ ]:
# S1: Get Shape Data
# download shapefile of states
gdf_states = (
    wr.s3.read_parquet(
        f"s3://{BUCKET}/{FOLDER}/{config['aws']['states-file']}"
    )
    # pass geometry column from binary to geometry
    .assign(geometry=lambda x: x['geometry'].apply(wkb.loads))
    .pipe(gpd.GeoDataFrame)
    .set_crs(epsg=4326)
    .rename(columns={"CODIGO": "cve_ent", "ESTADO": "cve_name"})
    .assign(
        cve_ent=lambda x: x['cve_ent'].str[-2:],
        cve_name=lambda x: x['cve_name'].str.lower()
    )
    .sort_values('cve_ent', ignore_index=True)
)

# S2: Wrangle Data
# get data of walmart stores
re_walmart = r"(walmart|wal mart|superama|waltmart)"
re_sams = r"(sams|sam's|sam s|sam's club|sam s club|sam'sclub|sam sclub|sam club|mi bodega)"
re_bodega = r"(bodega aurrera|bodega|aurrera|ba|boa|\$b|mb|b )"
re_supercenter = r"(supercenter|super center)"

# get walmart stores
gdf_walmart = (
    wr.s3.read_csv(
        f"s3://{BUCKET}/{FOLDER}/{config['aws']['walmart-file']}"
    )
    .assign(
        geometry=lambda x: gpd.points_from_xy(x['longitude'], x['latitude']),
        # get bodega aurrera or walmart or sams in name
        store_name=lambda x: np.select(
            [
                x['name'].str.contains(re_bodega, case=False),
                x['name'].str.contains(re_walmart, case=False),
                x['name'].str.contains(re_sams, case=False),
                x['name'].str.contains(re_supercenter, case=False)
            ],
            ['bodega aurrera', 'walmart', 'sams', 'supercenter'],
            default='other'
        )
    )
    .pipe(gpd.GeoDataFrame, crs="EPSG:4326")
    .query("store_name != 'other'")
    .loc[:, [
             'id', 'store_name', 'name', 'staff_stratum_description',
             'postal_code', 'cve_ent', 'cve_mun', 'geometry'
            ]]
)

In [ ]:
# S1: Get Connections Data
# get connections data from S3
gdf_connections = (
    wr.s3.read_parquet(
        f"s3://{BUCKET}/{FOLDER}/{config['aws']['connections-file']}"
    )
)

# S2: Wrangle Data
# obtain the day of the week of each test
gdf_connections['day_of_week'] = gdf_connections['result_date'].dt.day_name()

# Participación de mercado


In [ ]:
# Join Data
# join users with states
tbl_users_state = (
    gpd.sjoin_nearest(
        gdf_users.to_crs("EPSG:6372"), gdf_states.to_crs("EPSG:6372"),
    )
    .drop_duplicates(subset=["device_id"])
    .filter(["device_id", "postal_code", "raw_sim_operator_name", "cve_name"])
)

# define the dictionary
nielsen_zones = config['report']['nielsen_zones']

# map the states to their corresponding regions
tbl_users_state['region'] = (
    tbl_users_state['cve_name']
    .map(
        lambda x: next((region for region, states in nielsen_zones.items() if x.lower() in states), None)
        )
)

# droup by 'raw_sim_operator_name' and count 'device_id', then sort and get the top 10
top_operators = (
    tbl_users_state
    .groupby(["raw_sim_operator_name"])['device_id']
    .count()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
    ["raw_sim_operator_name"]
)

# Wrangle
tbl_users_state.loc[~tbl_users_state["raw_sim_operator_name"].isin(top_operators), "raw_sim_operator_name"] = "other"
# group by 'raw_sim_operator_name' and count 'device_id'
df_top_operator = tbl_users_state.groupby(["raw_sim_operator_name", "region"])['device_id'].count().reset_index()
# Get market share
df_top_operator["market_share"] = df_top_operator.groupby("region")["device_id"].transform(lambda x: x / x.sum())
# Pivot the DataFrame for the stacked bar chart
pivot_df = df_top_operator.pivot(index='region', columns='raw_sim_operator_name', values='market_share')
# Fill NaN values with 0
pivot_df = pivot_df.fillna(0)

# Plot
# stacked bar chart
pivot_df.plot(kind='bar', stacked=True, figsize=(12, 8), colormap='tab20')
# add labels and title
plt.xlabel('Estado')
plt.ylabel('Device ID Count')
plt.title('Top 10 SIM Operators by Device ID Count per Estado')
plt.legend(title='SIM Operator Name', bbox_to_anchor=(1.05, 1), loc='upper left')
# y lim
plt.ylim(0, 1)
# y in percentage
plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()])
# display
plt.tight_layout()
plt.show()

# Presencia nacional

In [ ]:
# Select only BAIT users
gdf_bait_users = gdf_users[gdf_users["raw_sim_operator_name"] == "BAIT"]

# Join users with stores getting the closest store
gdf_clients_stores = (
    gpd.sjoin_nearest(
        gdf_bait_users.to_crs("EPSG:6372"), gdf_walmart.to_crs("EPSG:6372"),
    )
    .drop_duplicates(subset=["device_id"])
    .groupby(["name", "store_name", "id", "cve_ent", "cve_mun"])
    .agg(
        count=("device_id", "count")
    )
    .reset_index()
    .sort_values("count", ascending=False)
    .merge(
        gdf_walmart.loc[:, ["id", "geometry"]],
        on="id",
        how="left"
    )
    .pipe(gpd.GeoDataFrame)
    .set_crs(epsg=4326)
)

# plot states with count of users
fig, ax = plt.subplots(figsize=(10, 10))
gdf_states.plot(ax=ax, color='gray', edgecolor='black')
gdf_clients_stores.plot(
    ax=ax,
    markersize=1,
    column='count',
    legend=True,
    cmap='viridis_r',
    alpha=0.1
)

# Análisis de tiendas por region


In [ ]:
# join users with states
gdf_clients_stores_region = (
    gpd.sjoin_nearest(
        gdf_clients_stores.to_crs("EPSG:6372"), gdf_states.to_crs("EPSG:6372"),
    )
)

# Map the states to their corresponding regions
gdf_clients_stores_region['region'] = (
    gdf_clients_stores_region['cve_name']
    .map(lambda x: 
    next((region for region, states in nielsen_zones.items() if x.lower() in states),None))
)

# Function to print the top 5 and bottom 5 per region
def top_bottom(region):
    stores = (gdf_clients_stores_region[gdf_clients_stores_region["region"]==region]
        .sort_values("count", ascending=False)
        )[["name", "store_name","cve_name","count"]]
    stores.columns = ["Nombre", "Formato", "Estado", "Clientes_reales"]

    ICD.display(f"Top 5 tiendas {region}")
    ICD.display(stores.head(n=5))
    ICD.display(f"Bottom 5 tiendas {region}")
    ICD.display(stores.tail(n=5))

for zone in nielsen_zones:
    top_bottom(zone)
    print("\n")

# Análisis temporal

# Distribución de usuarios por día de la semana

In [ ]:
# Obtain the number of users per day of the week
potential_users = (
    gdf_connections
    .groupby("day_of_week")
    .agg(
        count=("device_id", "count")
    )
    .reset_index()
)
# Sort the rows by day of the week
sorted_weekdays = config['report']['weekdays']
potential_users['day_of_week'] = pd.Categorical(potential_users['day_of_week'], sorted_weekdays)
potential_users = potential_users.sort_values("day_of_week")

# Obtain the number of BAIT users per day of the week
df_real_users = (
    gdf_connections
    [gdf_connections["raw_sim_operator_name"] == "BAIT"]
    .groupby("day_of_week")
    .agg(
        count=("device_id", "count")
    )
    .reset_index()
)
# Sort the rows by day of the week
df_real_users['day_of_week'] = pd.Categorical(df_real_users['day_of_week'], sorted_weekdays)
df_real_users = df_real_users.sort_values("day_of_week")

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 12))
# First plot
axes[0].bar(potential_users['day_of_week'], potential_users['count'])
axes[0].set_title('Número de usuarios por día de la semana')

# Second plot
axes[1].bar(df_real_users['day_of_week'], df_real_users['count'])
axes[1].set_title('Número de usuarios BAIT por día de la semana')

# Adjust layout
plt.tight_layout()
# Display
plt.show()

# Tienda A

In [ ]:
def store_user_analysis(postal_code: str) -> None:
    """
    Function to analyze the potential and real users of a store based on the postal code.
    ---
    postal_code: str
        Postal code of the store.
    """
    # Analyze the potential users of the store
    df_store_potential_users = (
        gdf_connections
        [gdf_connections["postal_code"] == postal_code]
        .groupby("day_of_week")
        .agg(
            count=("device_id", "count")
        )
        .reset_index()
    )

    # Sort the rows by day of the week
    df_store_potential_users['day_of_week'] = (
        pd.Categorical(df_store_potential_users['day_of_week'], sorted_weekdays)
        )
    df_store_potential_users = df_store_potential_users.sort_values("day_of_week")

    # Analyze the real users of the store
    df_store_real_users = (
        gdf_connections[
            (gdf_connections["raw_sim_operator_name"] == "BAIT") &
            (gdf_connections["postal_code"] == postal_code)
        ]
        .groupby("day_of_week")
        .agg(
            count=("device_id", "count")
        )
        .reset_index()
    )

    # Sort the rows by day of the week
    df_store_real_users['day_of_week'] = (pd
        .Categorical(df_store_real_users['day_of_week'], sorted_weekdays)
        )
    df_store_real_users = df_store_real_users.sort_values("day_of_week")

    # Plot
    # Comparison of potential users and real users
    fig, axes = plt.subplots(2, 1, figsize=(12, 12))

    # Distribution of the number of users per day of the week
    axes[0].bar(df_store_potential_users['day_of_week'], df_store_potential_users['count'])
    axes[0].set_title('Número de usuarios por día de la semana')

    # Distribution of the number of BAIT users per day of the week
    axes[1].bar(df_store_real_users['day_of_week'], df_store_real_users['count'])
    axes[1].set_title('Número de usuarios BAIT por día de la semana')

    # Adjust layout
    plt.tight_layout()

    # Display
    plt.show()

# Obtain the number of BAIT clients per store based in their postal_code
df_bait_users_per_store = (
    gdf_clients_stores_region
    .sort_values("count", ascending=False)
    [["name", "store_name","cve_name","count"]]
    )

# columns in spanish for better understanding
df_bait_users_per_store.columns = ["Nombre", "Formato", "Estado", "Clientes_reales"]

# Obtain the postal code of the most and less crowded store
top_postal_code = str(gdf_walmart[gdf_walmart["name"] == df_bait_users_per_store.head(n=1)["Nombre"][0]].postal_code.iloc[0])
bottom_postal_code = str(gdf_walmart[gdf_walmart["name"] == df_bait_users_per_store.tail(n=1)["Nombre"].iloc[0]].postal_code.iloc[0])

# User analysis for the most crowded store
store_user_analysis(top_postal_code)

## Tienda B

In [ ]:
# User analysis for the less crowded store
store_user_analysis(bottom_postal_code)